In [13]:
import os
import pandas as pd
import polars as pl
pd.set_option('display.max_rows', 100)
pl.Config.set_tbl_cols(100)
pl.Config.set_tbl_rows(100)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
from pathlib import Path

import polars as pl

from EventStream.data.dataset_polars import Dataset

DATA_DIR = Path("../MIMIC_ESD_new_schema_08-31-23-1")
ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.filter(~pl.all_horizontal(pl.all().is_null()))
dynamic_measurements_df = ESD.dynamic_measurements_df.filter(~pl.all_horizontal(pl.all().is_null()))

df_data = (events_df
            .join(dynamic_measurements_df, on="event_id", how="left")
            .drop(['event_id'])
            .sort(by=['subject_id', 'timestamp', 'event_type'])
            )


# df_data = pl.read_csv("sample_data.csv")

Updating config.save_dir from /n/data1/hms/dbmi/zaklab/RAMMS/data/MIMIC_IV/ESD_new_schema_08-31-23-1 to ../MIMIC_ESD_new_schema_08-31-23-1
Loading events from ../MIMIC_ESD_new_schema_08-31-23-1/events_df.parquet...


Loading dynamic_measurements from ../MIMIC_ESD_new_schema_08-31-23-1/dynamic_measurements_df.parquet...


In [15]:
def has_event_type(type_str: str) -> pl.Expr:
    has_event_type = pl.col("event_type").cast(pl.Utf8).str.contains(type_str)
    # has_event_type = event_types.str.contains(type_str)
    return has_event_type

# End-to-End

In [20]:
df_patient = df_data.filter(df_data['subject_id'] == 1790)
df_event = df_patient.filter(has_event_type("PROCEDURE_START") | has_event_type("PROCEDURE_END"))
# df_other = df_event.filter(df_event['procedure'].cast(pl.Utf8).str.contains("Invasive Ventilation"))
# df_hemoglobin = df_lab.filter(df_lab['lab'].cast(pl.Utf8).str.contains("Hemoglobin"))
df_event

subject_id,timestamp,event_type,age,time_of_day,measurement_id,admission_type,admission_location,language,race,marital_status,insurance,discharge_location,careunit,seq_num,diagnosis,text_value,valueuom,ref_range_lower,ref_range_upper,priority,flag,lab,valuenum,totalamountuom,rate,rateuom,infusion,totalamount,patientweight,procedure,medication
u16,datetime[μs],cat,f64,cat,u32,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,f32,f32,cat,cat,cat,f64,cat,f32,cat,cat,f64,f64,cat,cat
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",5791880,null,null,null,null,null,null,null,null,null,null,"""Continued""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",5791881,null,null,null,null,null,null,null,null,null,null,"""3 rails up""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",5791882,null,null,null,null,null,null,null,null,null,null,null,"""mmHg""",null,null,null,null,"""Arterial Blood…",108.0,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",41894944,null,null,null,null,null,null,null,null,null,null,"""3 rails up""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",41894945,null,null,null,null,null,null,null,null,null,null,"""Hazard-free en…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",41894946,null,null,null,null,null,null,null,null,null,null,"""Non-slip footw…",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",41894947,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",41894948,null,null,null,null,null,null,null,null,null,null,"""Patient""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1790,2126-07-21 03:15:00,"""LAB&PROCEDURE_…",28.547941,"""EARLY_AM""",77994736,null,null,null,null,null,null,null,null,null,null,"""45 Degrees""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [17]:
# from task_querying_v2 import *

# name = 'abnormal_lab'

# config_path = f"../sample_configs/{name}.yaml"

# res = query_task(config_path, df_data)
# res.filter(res['subject_id'] == 9011)


from task_querying_v2 import *

name = 'intervention_weaning'

config_path = f"../sample_configs/{name}.yaml"

res = query_task(config_path, df_data)
res.filter(res['subject_id'] == 1790)


# from task_querying_v2 import *

# name = 'readmission_risk'

# config_path = f"../sample_configs/{name}.yaml"

# res = query_task(config_path, df_data)
# res.filter(res['subject_id'] == 1984)

Loading config...

Generating predicate columns...



Added predicate column is_procedure_start.
Added predicate column is_procedure_end.
Added predicate column is_ventilation.
Added predicate column is_ventilation_start.
Added predicate column is_ventilation_end.

Building tree...
trigger
┣━━ target
┗━━ input


12127 subjects (14676057 rows) were excluded due to trigger condition: {'predicate': 'ventilation_start', 'min': 1, 'max': 1}.


Querying...


Querying subtree rooted at target...


subject_id,timestamp,timestamp_at_anchor,is_procedure_start,is_procedure_end,is_ventilation,is_ventilation_start,is_ventilation_end,is_any,is_procedure_start_summary,is_procedure_end_summary,is_ventilation_summary,is_ventilation_start_summary,is_ventilation_end_summary,is_any_summary
u16,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1790,2126-07-21 14:43:00,2126-07-21 03:15:00,2,2,0,0,1,57,0,0,0,0,0,0
1790,2126-08-12 14:20:00,2126-08-09 11:45:00,6,10,0,0,1,361,0,0,0,0,0,0
1790,2126-08-15 10:00:00,2126-08-14 15:10:00,3,4,0,0,1,141,0,0,0,0,0,0
1790,2126-09-02 04:06:00,2126-08-30 08:15:00,3,10,0,0,1,295,0,0,0,0,0,0
1790,2126-09-04 09:33:00,2126-09-04 09:33:00,0,0,0,0,0,0,0,0,0,0,0,0
1790,2126-10-07 11:11:00,2126-10-04 18:34:00,6,7,0,0,1,442,0,0,0,0,0,0


4261 subjects (5852 rows) were excluded due to constraint: [(col("is_ventilation_end")) >= (1)].
107 subjects (114 rows) were excluded due to constraint: [(col("is_ventilation_end")) <= (1)].


Querying subtree rooted at input...


subject_id,timestamp,timestamp_at_anchor,is_procedure_start,is_procedure_end,is_ventilation,is_ventilation_start,is_ventilation_end,is_any,is_procedure_start_summary,is_procedure_end_summary,is_ventilation_summary,is_ventilation_start_summary,is_ventilation_end_summary,is_any_summary
u16,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1790,2126-07-21 03:15:00,2126-07-21 03:15:00,1,0,1,1,0,6,0,0,0,0,0,0
1790,2126-08-09 11:45:00,2126-08-09 11:45:00,18,11,3,2,1,597,0,0,0,0,0,0
1790,2126-08-14 15:10:00,2126-08-14 15:10:00,27,23,5,3,2,1061,0,0,0,0,0,0
1790,2126-08-27 12:45:00,2126-08-27 12:45:00,29,26,5,3,2,1243,0,0,0,0,0,0
1790,2126-08-30 08:15:00,2126-08-30 08:15:00,38,31,6,4,2,1590,0,0,0,0,0,0
1790,2126-09-04 09:33:00,2126-09-04 09:33:00,49,46,8,5,4,2031,0,0,0,0,0,0
1790,2126-10-04 18:34:00,2126-10-04 18:34:00,6,7,1,1,0,260,0,0,0,0,0,0


6820 subjects (8719 rows) were excluded due to constraint: [(col("is_any")) >= (100)].


Done.



subject_id,trigger/timestamp,target/timestamp,input/timestamp,target/window_summary,input/window_summary
u16,datetime[μs],datetime[μs],datetime[μs],struct[6],struct[6]
1790,2126-08-09 11:45:00,2126-08-12 14:20:00,2126-07-10 11:45:00,"{6,10,0,0,1,361}","{18,11,3,2,1,597}"
1790,2126-08-14 15:10:00,2126-08-15 10:00:00,2126-07-15 15:10:00,"{3,4,0,0,1,141}","{27,23,5,3,2,1061}"
1790,2126-08-30 08:15:00,2126-09-02 04:06:00,2126-07-31 08:15:00,"{3,10,0,0,1,295}","{38,31,6,4,2,1590}"
1790,2126-10-04 18:34:00,2126-10-07 11:11:00,2126-09-04 18:34:00,"{6,7,0,0,1,442}","{6,7,1,1,0,260}"


In [19]:
res.filter(res['subject_id'] == 1790).select(['target/window_summary']).unnest('target/window_summary')

is_procedure_start,is_procedure_end,is_ventilation,is_ventilation_start,is_ventilation_end,is_any
i32,i32,i32,i32,i32,i32
6,10,0,0,1,361
3,4,0,0,1,141
3,10,0,0,1,295
6,7,0,0,1,442


# Profiling

In [ ]:
import cProfile, pstats, sys
from task_querying_v2 import *

name = 'profiling'
config_path = f"test_configs/{name}.yaml"

total_rows = [10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 5000000, 10000000, 50000000, 100000000, 150000000]

for i in total_rows:
    df_temp = df_data.head(i)
    print(f"Number of rows: {df_temp.shape[0]}")
    print(f"Number of patients: {df_temp['subject_id'].n_unique()}")

    pr = cProfile.Profile()
    pr.enable()

    df_result = query_task(config_path, df_temp, verbose=False)

    pr.disable()
    ps = pstats.Stats(pr, stream=sys.stdout)
    ps.sort_stats("cumtime").print_stats()

    print(f"Number of rows: {df_result.shape[0]}")
    print(f"Number of patients: {df_result['subject_id'].n_unique()}")
    print("=====================================NEXT=====================================")

Number of rows: 10
Number of patients: 1
         175709 function calls (173457 primitive calls) in 0.350 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.350    0.350 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v2.py:818(query_task)
     11/1    0.016    0.001    0.267    0.267 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v2.py:543(query_subtree)
      179    0.001    0.000    0.217    0.001 /home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/polars/lazyframe/frame.py:1814(collect)
      179    0.205    0.001    0.205    0.001 {method 'collect' of 'builtins.PyLazyFrame' objects}
       34    0.001    0.000    0.149    0.004 /home/justinxu/miniconda3/envs/esgpt/lib/python3.10/site-packages/polars/dataframe/frame.py:6305(join)
       10    0.018    0.002    0.124    0.012 /home/justinxu/esgpt/ESGPTTaskQuerying/to_organize/task_querying_v

KeyboardInterrupt: 

# Unit Testing

In [ ]:
from simple_test_runner import *

In [ ]:
df_test_1_input = pl.DataFrame(
    {
        'subject_id': [1, 1, 1],
        'timestamp': ['12/1/1900 12:00', '12/1/1900 13:00', '12/1/1900 14:00'],
        'event_type': ['ADMISSION', 'LAB', 'DISCHARGE'],
        'dx': ['', '', ''],
        'lab_test': ['', 'SpO2', ''],
        'lab_value': ['', '99', ''],
        'is_admission': [1, 0, 0],
        'is_lab': [0, 1, 0],
        'is_discharge': [0, 0, 1],
        'is_any': [1, 1, 1],
    }
)

df_test_1_predicate_cols = [col for col in df_test_1_input.columns if col.startswith('is_')]

df_test_1_endpoint_expr = (False, "is_discharge", True, None) 

test_1_anchor_to_subtree_root_by_subtree_anchor = pl.DataFrame(
    {
        'subject_id': [1],
        'timestamp': ['12/1/1900 12:00'],
        'is_admission': [0],
        'is_lab': [0],
        'is_discharge': [0],
        'is_any': [0],
    }
)

df_test_1_result = pl.DataFrame(
    {
        "subject_id": [1],
        "timestamp": ["12/1/1900 14:00"],
        "timestamp_at_anchor": ["12/1/1900 12:00"],
        "is_admission": [0],
        "is_lab": [1],
        "is_discharge": [1],
        "is_any": [2],
        "is_admission_summary": [0],
        "is_lab_summary": [0],
        "is_discharge_summary": [0],
        "is_any_summary": [0],
    }
)

In [ ]:
test_1 = [
    {
        "message": "Testing summarize_event_bound_window, should be equal...",
        "args": (
            df_test_1_input, 
            df_test_1_predicate_cols, 
            df_test_1_endpoint_expr,
            test_1_anchor_to_subtree_root_by_subtree_anchor),
        "want": df_test_1_result,
    }
]

simple_test_runner(test_1, summarize_event_bound_window)

Testing summarize_event_bound_window, should be equal... Passed!


# Integration Testing

### Config Loading

In [ ]:
from task_querying_v2 import *

In [ ]:
data = pl.read_csv('sample_data.csv')
data = data.with_columns(pl.col('timestamp').str.strptime(pl.Datetime, format='%m/%d/%Y %H:%M').cast(pl.Datetime))

In [ ]:
cfg = load_config('sample_config.yaml')
cfg

{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Generate predicate columns

In [ ]:
predicates_df = generate_predicate_columns(cfg, data)
predicates_df

AttributeError: 'DotAccessibleDict' object has no attribute 'system'

### Build tree

In [ ]:
tree = build_tree_from_config(cfg)
print_tree(tree, style="const_bold")
for each_node in preorder_iter(tree):
    print(each_node)
cfg

trigger
┣━━ gap
┃   ┗━━ target
┗━━ input
Node(/trigger, constraints={'is_admission': (1, 1)}, endpoint_expr=(False, 'is_admission', True, datetime.timedelta(0)))
Node(/trigger/gap, constraints={'is_admission': (None, 0), 'is_discharge': (None, 0), 'is_death': (None, 0)}, endpoint_expr=(False, datetime.timedelta(days=2), True, datetime.timedelta(0)))
Node(/trigger/gap/target, constraints={'is_discharge_or_death': (1, 1)}, endpoint_expr=(True, 'is_discharge_or_death', True, datetime.timedelta(0)))
Node(/trigger/input, constraints={'is_covid': (None, 0), 'is_any': (5, None)}, endpoint_expr=(False, datetime.timedelta(days=30), True, datetime.timedelta(days=1)))


{'predicates': {'admission': {'column': 'event_type', 'value': 'ADMISSION'},
  'discharge': {'column': 'event_type', 'value': 'DISCHARGE'},
  'death': {'column': 'event_type', 'value': 'DEATH'},
  'covid': {'column': 'dx', 'value': 'COVID'},
  'discharge_or_death': {'type': 'ANY', 'predicates': ['discharge', 'death']}},
 'windows': {'trigger': {'start': 'admission',
   'duration': None,
   'offset': None,
   'end': 'admission',
   'excludes': None,
   'includes': [{'predicate': 'admission', 'min': 1, 'max': 1}],
   'st_inclusive': False,
   'end_inclusive': True},
  'gap': {'start': 'trigger.end',
   'duration': '48 hours',
   'offset': None,
   'end': 'gap.start + gap.duration',
   'excludes': [{'predicate': 'admission'}, {'predicate': 'discharge'}, {'predicate': 'death'}],
   'includes': None,
   'st_inclusive': False,
   'end_inclusive': True},
  'target': {'start': 'gap.end',
   'duration': None,
   'offset': None,
   'end': 'discharge_or_death',
   'excludes': None,
   'includes':

### Summarize window

In [ ]:
import datetime

predicate_cols = [col for col in predicates_df.columns if col.startswith("is_")]
print(predicate_cols)

anchor_to_subtree_root_by_subtree_anchor = (
        predicates_df.filter(predicates_df['is_admission'] == 1)
            .select('subject_id', 'timestamp', *[pl.col(c) for c in predicate_cols])
            .with_columns('subject_id', 'timestamp', *[pl.lit(0).alias(c) for c in predicate_cols])
    )
print(anchor_to_subtree_root_by_subtree_anchor)

['is_admission', 'is_discharge', 'is_death', 'is_covid', 'is_discharge_or_death', 'is_any']
shape: (6, 8)
┌────────────┬─────────────┬─────────────┬─────────────┬──────────┬──────────┬────────────┬────────┐
│ subject_id ┆ timestamp   ┆ is_admissio ┆ is_discharg ┆ is_death ┆ is_covid ┆ is_dischar ┆ is_any │
│ ---        ┆ ---         ┆ n           ┆ e           ┆ ---      ┆ ---      ┆ ge_or_deat ┆ ---    │
│ i64        ┆ datetime[μs ┆ ---         ┆ ---         ┆ i32      ┆ i32      ┆ h          ┆ i32    │
│            ┆ ]           ┆ i32         ┆ i32         ┆          ┆          ┆ ---        ┆        │
│            ┆             ┆             ┆             ┆          ┆          ┆ i32        ┆        │
╞════════════╪═════════════╪═════════════╪═════════════╪══════════╪══════════╪════════════╪════════╡
│ 1          ┆ 1989-12-01  ┆ 0           ┆ 0           ┆ 0        ┆ 0        ┆ 0          ┆ 0      │
│            ┆ 12:03:00    ┆             ┆             ┆          ┆          ┆        

Temporally-bound

In [ ]:
summarize_temporal_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, timedelta(days=2), True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-01 12:03:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-27 23:32:00,1991-01-27 23:32:00,0,0,0,0,0,3,0,0,0,0,0,0
1,1991-03-03 19:33:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 02:24:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-05 00:32:00,1996-06-05 00:32:00,0,0,0,0,0,0,0,0,0,0,0,0
3,1996-03-08 02:24:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0


Event-bound

In [ ]:
summarize_event_bound_window(predicates_df=predicates_df, predicate_cols=predicate_cols, endpoint_expr=(False, 'is_discharge_or_death', True, None), anchor_to_subtree_root_by_subtree_anchor=anchor_to_subtree_root_by_subtree_anchor)

subject_id,timestamp,timestamp_at_anchor,is_admission,is_discharge,is_death,is_covid,is_discharge_or_death,is_any,is_admission_summary,is_discharge_summary,is_death_summary,is_covid_summary,is_discharge_or_death_summary,is_any_summary
i64,datetime[μs],datetime[μs],i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1,1989-12-02 15:00:00,1989-12-01 12:03:00,0,1,0,0,1,7,0,0,0,0,0,0
1,1991-01-31 02:15:00,1991-01-27 23:32:00,0,1,0,0,1,9,0,0,0,0,0,0
1,1991-03-03 21:38:00,1991-03-03 19:33:00,0,0,1,0,1,2,0,0,0,0,0,0
2,1996-03-08 16:00:00,1996-03-08 02:24:00,0,1,0,1,1,4,0,0,0,0,0,0
2,1996-06-08 03:00:00,1996-06-05 00:32:00,0,0,1,0,1,5,0,0,0,0,0,0
3,1996-03-10 00:00:00,1996-03-08 02:24:00,0,0,1,0,1,6,0,0,0,0,0,0
